<a href="https://colab.research.google.com/github/LeBanjoKazooie/VirtualHumanProject/blob/main/Copy_of_VHPTeam3v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project was developed by COMP 670 Team 3 Virtual Human Project and it is based on the project specifications from Professor Chu. The goal of the project is to develop a Virtual Human Advisor for Franklin University. This app was initially developed using streamlit and replicate documentation. Later it was ported to Google Colab using HuggingFace Llama2 and Transformers documentation. While re-running the application Google Colab Help, Warnings, and Examples were useful in adding Google Drive, swapfile support and making minor edits to improve the application. The LLM model used is Lama-2-7b-chat-hf which uses the LamaForCausalLM architecture and dtype of Float16.

In [ ]:
%pip install llama-index
%pip install transformers
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install llama-index-readers-web
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-program-openai
%pip install llama-index-agent-openai
%pip install InstructorEmbedding
%pip install torch
%pip install pytorch-cuda
%pip install safetensors
%pip install google.colab
%pip install sentence transformers
%pip install accelerate

In [ ]:
from llama_index.core import SimpleDirectoryReader
from google.colab import userdata
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
from InstructorEmbedding import INSTRUCTOR
from llama_index.core import VectorStoreIndex
from llama_index.core.indices import SummaryIndex
from llama_index.core.response.notebook_utils import display_response
import logging
import sys
import safetensors
from torch.cuda import is_available
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import LlamaForCausalLM, LlamaConfig, LlamaTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from llama_index.core.evaluation import EvaluationResult
from llama_index.core.node_parser import TextSplitter
from llama_index.core.node_parser import SentenceSplitter
import accelerate

LOADING DOCUMENTS:
Create a folder named data next to the sample_data folder and upload the pdf documents to the folder data

In [ ]:
documents = SimpleDirectoryReader("/content/data").load_data()

In [ ]:
hf_token = userdata.get('hf_token')

SWAPFILE in progress

In [ ]:
!fallocate -l 64G /content/swapfile
!chmod 600 /content/swapfile
!mkswap /content/swapfile
!swapon --all --verbose /content/swapfile
!swapon --show
!free -h

LOADING LlamaForCasualLM model with offload

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
config = LlamaConfig.from_pretrained(model_name, token=hf_token, use_cache=True)

with init_empty_weights():
  model = AutoModelForCausalLM.from_config(config)

device_map = infer_auto_device_map(model)

if torch.cuda.is_available():
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.float16,
      #bnb_4bit_quant_type="nf4",
      #bnb_4bit_use_double_quant=True,
  )
  model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto", token=hf_token, offload_folder="llama-disk-offload", use_cache=True, quantization_config=quantization_config)
else:
  model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto", token=hf_token, offload_folder="llama-disk-offload", use_cache=True)


CONFIGURING HuggingFaceLLM MODEL FROM LlamaForCasualLM model: Load Llm model configuration. The maximum number of tokens on the input context is 2000.  The maximum number of response tokens is set to 512.

In [ ]:
if torch.cuda.is_available():
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.float16,
      #bnb_4bit_quant_type="nf4",
      #bnb_4bit_use_double_quant=True,
  )
  hf_model = HuggingFaceLLM(
      model = model,
      model_name=model_name,
      tokenizer_name=model_name,
      query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
      context_window=1024,
      model_kwargs={"token": hf_token, "quantization_config": quantization_config},
      tokenizer_kwargs={"token": hf_token},
      device_map="auto",
      max_new_tokens=512,
  )
else:
  hf_model = HuggingFaceLLM(
      model = model,
      model_name=model_name,
      tokenizer_name=model_name,
      query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
      context_window=1024,
      tokenizer_kwargs={"token": hf_token},
      device_map="auto",
      max_new_tokens=512,
  )

INDEXING:
Generate Vector index, embedings and transformations for Vector Search

In [ ]:
llama_embed_model = 'text-embedding-ada-002'
embed_model = 'local:hkunlp/instructor-large'
Settings.llm = hf_model

if torch.cuda.is_available():
  Settings.chunk_size = 512
  Settings.chunk_overlap = 20
  Settings.text_splitter = SentenceSplitter(chunk_size=512)
  Settings.transformations = [Settings.text_splitter]
else:
  Settings.chunk_size = 512
  Settings.chunk_overlap = 20
  Settings.text_splitter = SentenceSplitter(chunk_size=512)
  Settings.transformations = [Settings.text_splitter]

Settings.embed_model=embed_model
embeddings = Settings.embed_model(documents)

print(f"Embeddings: {embeddings}")
transformations = Settings.transformations
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, transformations=transformations)

STORING in progress:
Store vector embeddings and indexing

QUERING: The chatbot is primarily used for questions about Franklin University.

In [ ]:
#query_engine = vector_index.as_query_engine(response_mode="compact")
#prompt = str(input("Ask me a question about Franklin University!  "))
#response = query_engine.query("What are Ohio CPA Examination requirements?")

chat_engine = vector_index.as_chat_engine(chat_mode="context",response_mode="compact",max_new_tokens=512,
                                          system_prompt=("You are a chatbot, able to have normal interactions, as well as talk about Franklin University"),
                                          llm=hf_model
                                          )
prompt = str(input("Ask me a question about Franklin University!  "))
response = chat_engine.chat(prompt)

display_response(response)

EVALUATION in progress:
Store querying prompt and responses for evaluation.

In [ ]:
!pip install llama-index-vector-stores-qdrant qdrant_client
!pip install fastembed


In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from llama_index.core import StorageContext, VectorStoreIndex

qdrant_client = QdrantClient(
    url="https://4f411c47-4cba-4085-810c-dafbb4ca3bc3.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="b9cWve2imbA9NJafCxUmiAAevxyjgcbns67-LRPhmTtSAkUf38qHxw",
)

vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="mycollection",
    enable_hybrid=True,
    batch_size=20
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context
)



In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

qdrant_client = QdrantClient(
    url="https://4f411c47-4cba-4085-810c-dafbb4ca3bc3.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="b9cWve2imbA9NJafCxUmiAAevxyjgcbns67-LRPhmTtSAkUf38qHxw",
)

vector_store = QdrantVectorStore(client=qdrant_client, collection_name="mycollection", enable_hybrid=True)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
query_engine = index.as_query_engine(response_mode="compact")
prompt = str(input("Ask me a question about Franklin University!  "))
response = query_engine.query(prompt)

display_response(response)

In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile app.py

import streamlit as st

st.write('Hello, *World!* :sunglasses:')

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!npx localtunnel --port 8501
